In [1]:
# # Change your working directory to where the data resides1Q
import os
os.chdir('/home/rene/Capstone2 b/Data')
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
import dask.array as da
import xgboost as xgb
import dask_ml
from dask_ml.preprocessing import Categorizer, DummyEncoder
from dask_ml.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from dask_ml.wrappers import ParallelPostFit
from dask_ml.preprocessing import StandardScaler

pd.options.display.max_columns= None
pd.options.display.max_rows = None

pd.options.display.width=None

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=1, threads_per_worker=12)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:38877 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 12 Memory: 33.52 GB


In [2]:
%%time
X_test = dd.read_parquet('X_test.parquet.gzip', engine = 'pyarrow', chunksize=700)

CPU times: user 13 ms, sys: 428 µs, total: 13.4 ms
Wall time: 12.1 ms


In [3]:
%%time
X_test.head()

CPU times: user 22.1 ms, sys: 2.11 ms, total: 24.2 ms
Wall time: 442 ms


,msno
462124,VVI6bwi0/CAD6vtBvP1P7CwAW8icAn2k+uHBao2mt5A=
841807,cRWeTCtcxwX93NCJ8ZI925ag1yCE0QrhqHLRlGPx3mw=
383244,y1pFnxTeM6gTsELJfwPJd5vzBPW1LB8mGBbRcu/h1tY=
781796,gYpbJguK8xFMGGxT1CPz9NyGQb4waENHftfXE5T3Huc=
312155,N8tvTeloNBV6mg95W1i9CqWlCGVY2m3XktuzPjjr+BQ=


In [4]:
%%time
X_test = X_test.merge(dd.read_parquet('y_test.parquet.gzip', engine='pyarrow', chunksize=700),left_index=True, right_index=True)
X_test.head()

CPU times: user 19.2 ms, sys: 1.33 ms, total: 20.6 ms
Wall time: 133 ms


,msno,is_churn
462124,VVI6bwi0/CAD6vtBvP1P7CwAW8icAn2k+uHBao2mt5A=,0
841807,cRWeTCtcxwX93NCJ8ZI925ag1yCE0QrhqHLRlGPx3mw=,0
383244,y1pFnxTeM6gTsELJfwPJd5vzBPW1LB8mGBbRcu/h1tY=,0
781796,gYpbJguK8xFMGGxT1CPz9NyGQb4waENHftfXE5T3Huc=,0
312155,N8tvTeloNBV6mg95W1i9CqWlCGVY2m3XktuzPjjr+BQ=,0


In [5]:
%%time
X_test =X_test.merge(dd.read_parquet('test_members_logs_trans.parquet.gzip', engine='pyarrow', chunksize=700), on='msno')
X_test

CPU times: user 50 ms, sys: 4.98 ms, total: 55 ms
Wall time: 50.6 ms


,msno,is_churn,city,bd,gender,registered_via,registration_init_time,min_payment_id,max_payment_id,mean_payment_id,std_payment_id,sum_payment_id,min_plan_days,max_plan_days,mean_plan_days,std_plan_days,sum_plan_days,min_price_list,max_price_list,mean_price_list,std_price_list,sum_price_list,min_amount_paid,max_amount_paid,mean_amount_paid,std_amount_paid,sum_amount_paid,max_renew,count_renew,min_transaction_date,max_transaction_date,count_transaction_date,min_expire_date,max_expire_date,max_is_cancel,count_is_cancel,max_auto_renew_&_cancel,count_auto_renew_&_cancel,min_days_to_expire,max_days_to_expire,mean_days_to_expire,std_days_to_expire,sum_days_to_expire,min_unpaid,max_unpaid,mean_unpaid,std_unpaid,sum_unpaid,min_pmethod_renew,max_pmethod_renew,mean_pmethod_renew,std_pmethod_renew,sum_pmethod_renew,min_date,max_date,count_date,min_num_25,max_num_25,mean_num_25,std_num_25,sum_num_25,min_num_50,max_num_50,mean_num_50,std_num_50,sum_num_50,min_num_75,max_num_75,mean_num_75,std_num_75,sum_num_75,min_num_985,max_num_985,mean_num_985,std_num_985,sum_num_985,min_num_100,max_num_100,mean_num_100,std_num_100,sum_num_100,min_num_unq,max_num_unq,mean_num_unq,std_num_unq,sum_num_unq,min_total_secs,max_total_secs,mean_total_secs,std_total_secs,sum_total_secs,min_skip_ratio,max_skip_ratio,mean_skip_ratio,std_skip_ratio,sum_skip_ratio,min_skip25_ratio,max_skip25_ratio,mean_skip25_ratio,std_skip25_ratio,sum_skip25_ratio,min_skip50_ratio,max_skip50_ratio,mean_skip50_ratio,std_skip50_ratio,sum_skip50_ratio,min_skip75_ratio,max_skip75_ratio,mean_skip75_ratio,std_skip75_ratio,sum_skip75_ratio,min_skip985_ratio,max_skip985_ratio,mean_skip985_ratio,std_skip985_ratio,sum_skip985_ratio,min_num100_ratio,max_num100_ratio,mean_num100_ratio,std_num100_ratio,sum_num100_ratio,min_unq_ratio,max_unq_ratio,mean_unq_ratio,std_unq_ratio,sum_unq_ratio,min_unq_secs_ratio,max_unq_secs_ratio,mean_unq_secs_ratio,std_unq_secs_ratio,sum_unq_secs_ratio,min_num100_secs_ratio,max_num100_secs_ratio,mean_num100_secs_ratio,std_num100_secs_ratio,sum_num100_secs_ratio,min_skip_secs_ratio,max_skip_secs_ratio,mean_skip_secs_ratio,std_skip_secs_ratio,sum_skip_secs_ratio,min_skip25_secs_ratio,max_skip25_secs_ratio,mean_skip25_secs_ratio,std_skip25_secs_ratio,sum_skip25_secs_ratio,min_skip50_secs_ratio,max_skip50_secs_ratio,mean_skip50_secs_ratio,std_skip50_secs_ratio,sum_skip50_secs_ratio,min_skip75_secs_ratio,max_skip75_secs_ratio,mean_skip75_secs_ratio,std_skip75_secs_ratio,sum_skip75_secs_ratio,min_skip985_secs_ratio,max_skip985_secs_ratio,mean_skip985_secs_ratio,std_skip985_secs_ratio,sum_skip985_secs_ratio,min_daily_listening_ratio,max_daily_listening_ratio,mean_daily_listening_ratio,std_daily_listening_ratio,sum_daily_listening_ratio,registration_init_year,registration_init_month,registration_init_day,max_transaction_year,max_transaction_month,max_transaction_day,account_age
npartitions=2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,int64,int64,int64,object,int64,datetime64[ns],int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,datetime64[ns],datetime64[ns],int64,datetime64[ns],datetime64[ns],int64,int64,int64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,datetime64[ns],datetime64[ns],int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64

In [6]:
%%time
X_test.head()

CPU times: user 35.1 ms, sys: 11.1 ms, total: 46.2 ms
Wall time: 1.73 s


,msno,is_churn,city,bd,gender,registered_via,registration_init_time,min_payment_id,max_payment_id,mean_payment_id,std_payment_id,sum_payment_id,min_plan_days,max_plan_days,mean_plan_days,std_plan_days,sum_plan_days,min_price_list,max_price_list,mean_price_list,std_price_list,sum_price_list,min_amount_paid,max_amount_paid,mean_amount_paid,std_amount_paid,sum_amount_paid,max_renew,count_renew,min_transaction_date,max_transaction_date,count_transaction_date,min_expire_date,max_expire_date,max_is_cancel,count_is_cancel,max_auto_renew_&_cancel,count_auto_renew_&_cancel,min_days_to_expire,max_days_to_expire,mean_days_to_expire,std_days_to_expire,sum_days_to_expire,min_unpaid,max_unpaid,mean_unpaid,std_unpaid,sum_unpaid,min_pmethod_renew,max_pmethod_renew,mean_pmethod_renew,std_pmethod_renew,sum_pmethod_renew,min_date,max_date,count_date,min_num_25,max_num_25,mean_num_25,std_num_25,sum_num_25,min_num_50,max_num_50,mean_num_50,std_num_50,sum_num_50,min_num_75,max_num_75,mean_num_75,std_num_75,sum_num_75,min_num_985,max_num_985,mean_num_985,std_num_985,sum_num_985,min_num_100,max_num_100,mean_num_100,std_num_100,sum_num_100,min_num_unq,max_num_unq,mean_num_unq,std_num_unq,sum_num_unq,min_total_secs,max_total_secs,mean_total_secs,std_total_secs,sum_total_secs,min_skip_ratio,max_skip_ratio,mean_skip_ratio,std_skip_ratio,sum_skip_ratio,min_skip25_ratio,max_skip25_ratio,mean_skip25_ratio,std_skip25_ratio,sum_skip25_ratio,min_skip50_ratio,max_skip50_ratio,mean_skip50_ratio,std_skip50_ratio,sum_skip50_ratio,min_skip75_ratio,max_skip75_ratio,mean_skip75_ratio,std_skip75_ratio,sum_skip75_ratio,min_skip985_ratio,max_skip985_ratio,mean_skip985_ratio,std_skip985_ratio,sum_skip985_ratio,min_num100_ratio,max_num100_ratio,mean_num100_ratio,std_num100_ratio,sum_num100_ratio,min_unq_ratio,max_unq_ratio,mean_unq_ratio,std_unq_ratio,sum_unq_ratio,min_unq_secs_ratio,max_unq_secs_ratio,mean_unq_secs_ratio,std_unq_secs_ratio,sum_unq_secs_ratio,min_num100_secs_ratio,max_num100_secs_ratio,mean_num100_secs_ratio,std_num100_secs_ratio,sum_num100_secs_ratio,min_skip_secs_ratio,max_skip_secs_ratio,mean_skip_secs_ratio,std_skip_secs_ratio,sum_skip_secs_ratio,min_skip25_secs_ratio,max_skip25_secs_ratio,mean_skip25_secs_ratio,std_skip25_secs_ratio,sum_skip25_secs_ratio,min_skip50_secs_ratio,max_skip50_secs_ratio,mean_skip50_secs_ratio,std_skip50_secs_ratio,sum_skip50_secs_ratio,min_skip75_secs_ratio,max_skip75_secs_ratio,mean_skip75_secs_ratio,std_skip75_secs_ratio,sum_skip75_secs_ratio,min_skip985_secs_ratio,max_skip985_secs_ratio,mean_skip985_secs_ratio,std_skip985_secs_ratio,sum_skip985_secs_ratio,min_daily_listening_ratio,max_daily_listening_ratio,mean_daily_listening_ratio,std_daily_listening_ratio,sum_daily_listening_ratio,registration_init_year,registration_init_month,registration_init_day,max_transaction_year,max_transaction_month,max_transaction_day,account_age
0,VVI6bwi0/CAD6vtBvP1P7CwAW8icAn2k+uHBao2mt5A=,0,1,0,None,7,2016-11-17,41,41,41.0,0.0,205,30,30,30.0,0.0,150,99,99,99.0,0.0,495,99,99,99.0,0.0,495,1,5,2016-11-17,2017-03-16,5,2016-12-16,2017-04-16,0,5,0,5,-335,31,-43.200000,163.126331,-216,0,0,0.0,0.0,0,41,41,41.0,0.0,205,2016-11-17,2017-03-30,104,0,65,12.086538,13.480738,1257,0,8,1.490385,1.779032,155,0,6,0.990385,1.361718,103,0,7,1.221154,1.612793,127,0,145,38.605769,28.657813,4015,1,158,46.346154,31.782260,4820,2.254,34230.405,10079.793779,7249.300566,1048298.553,0.0,1.00,0.276985,0.222647,28.806413,0.0,1.000000,0.198420,0.181289,20.635700,0.0,1.000000,0.036673,0.101606,3.813943,0.0,0.10,0.015096,0.020504,1.570011,0.0,0.500000,0.026796,0.057312,2.786758,0.00,1.0,0.723015,0.222647,75.193587,0.441176,1.0,0.891865,0.115465,92.753979,2.25400,33592.571366,8602.502113,6085.640320,8.946602e+05,0.0,4.963409e+06,5.936608e+05,7.689261e+05,6.174072e+07,0.0,10274.112654,2570.802344,2280.631236,2.673634e+05,0.0,9148.182500,1930.994680,1870.912680,2.008234e+05,0.0,1285.192000,255.669790,304.545003,26589.658199,0.0,1038.914345,176.486098,241.668483,18354.554216

In [7]:
X_test['gender'] = X_test['gender'].astype('category') 
X_test['gender'] = X_test['gender'].cat.add_categories('not_specified')
X_test['gender'] = X_test['gender'].fillna('not_specified')
X_test['gender'].head()


0    not_specified
1    not_specified
2    not_specified
3           female
4           female
Name: gender, dtype: category
Categories (3, object): ['female', 'male', 'not_specified']

In [8]:
%%time

X_train = dd.read_parquet('X_train.parquet.gzip', engine = 'pyarrow', chunksize=700)

X_train = X_train.merge(dd.read_parquet('y_train.parquet.gzip', engine='pyarrow', chunksize=700),left_index=True, right_index=True)

X_train =X_train.merge(dd.read_parquet('train_members_logs_trans.parquet.gzip', engine='pyarrow', chunksize=700), on='msno')


X_train['gender'] = X_train['gender'].astype('category')
X_train['gender'] = X_train['gender'].cat.add_categories('not_specified')
X_train['gender'] = X_train['gender'].fillna('not_specified')
display(X_train['gender'].head())

0    not_specified
1    not_specified
2    not_specified
3             male
4           female
Name: gender, dtype: category
Categories (3, object): ['female', 'male', 'not_specified']

CPU times: user 213 ms, sys: 30.6 ms, total: 243 ms
Wall time: 7.7 s


In [9]:
%%time

de = DummyEncoder()

X = X_train.drop(columns=["msno", "is_churn", "registration_init_time", "min_date", "max_date", "min_transaction_date", "max_transaction_date", "min_expire_date", "max_expire_date"])
Y =  X_train["is_churn"]

X_eval =  X_test.drop(columns=["msno", "is_churn", "registration_init_time", "min_date", "max_date", "min_transaction_date", "max_transaction_date", "min_expire_date", "max_expire_date"])
y_eval = X_test['is_churn']


X = X.categorize()
data = de.fit_transform(X)

labels = Y
X_eval = X_eval.categorize()
X_eval = de.transform(X_eval)

CPU times: user 325 ms, sys: 43.3 ms, total: 369 ms
Wall time: 12.7 s


In [10]:
data.isna().sum().compute()

city                             0
bd                               0
registered_via                   0
min_payment_id                   0
max_payment_id                   0
mean_payment_id                  0
std_payment_id                5677
sum_payment_id                   0
min_plan_days                    0
max_plan_days                    0
mean_plan_days                   0
std_plan_days                 5677
sum_plan_days                    0
min_price_list                   0
max_price_list                   0
mean_price_list                  0
std_price_list                5677
sum_price_list                   0
min_amount_paid                  0
max_amount_paid                  0
mean_amount_paid                 0
std_amount_paid               5677
sum_amount_paid                  0
max_renew                        0
count_renew                      0
count_transaction_date           0
max_is_cancel                    0
count_is_cancel                  0
max_auto_renew_&_can

In [11]:
%%time

data_1 = data.fillna(value=0)

scaler = StandardScaler()

data_1 = scaler.fit_transform(data_1)

CPU times: user 273 ms, sys: 44.2 ms, total: 317 ms
Wall time: 11.8 s


In [12]:
%%time
clf = ParallelPostFit(LogisticRegressionCV(cv=3,  max_iter=1000), scoring="accuracy")
clf.fit(data_1, labels)

CPU times: user 1h 3min 57s, sys: 17min 26s, total: 1h 21min 24s
Wall time: 7min 21s


ParallelPostFit(estimator=LogisticRegressionCV(cv=3, max_iter=1000),
                scoring='accuracy')

In [13]:
%%time
X_eval_1 = X_eval.fillna(value=0)
X_eval_1 = scaler.transform(X_eval_1)
clf.score(X_eval_1, y_eval)

CPU times: user 513 ms, sys: 529 ms, total: 1.04 s
Wall time: 2.92 s


0.9441854752393597

### XGboost - Gradient Boost Tree model

In [14]:
%%time
pdtrain = xgb.dask.DaskDMatrix(client, data, label=labels, feature_names=data.columns)
dval = xgb.dask.DaskDMatrix(client, X_eval, label=y_eval, feature_names=X_eval.columns)

CPU times: user 472 ms, sys: 60.3 ms, total: 532 ms
Wall time: 22.4 s


In [15]:
%%time
# Use train method from xgboost.dask instead of xgboost.  This
# distributed version of train returns a dictionary containing the
# resulting booster and evaluation history obtained from
# evaluation metrics.
output = xgb.dask.train(client,
                        {'verbosity': 1,
                         "objective": "binary:logistic",
                        "eval_metric": ["error"],
                        'tree_method': 'hist'},
                        pdtrain,
                        num_boost_round=3000,
                        evals=[(dval, 'valid')],
                        early_stopping_rounds=10)
bst = output['booster']
history = output['history']

# you can pass output directly into `predict` too.
prediction = xgb.dask.predict(client, bst, dval)
print('Evaluation history:', history)
print('Best iteration:', bst.best_iteration)
print ('Best score:', bst.best_score)

/home/rene/miniconda3/lib/python3.8/site-packages/distributed/client.py:3530: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)


Evaluation history: {'valid': {'error': [0.04741, 0.046252, 0.045842, 0.045141, 0.044667, 0.04431, 0.042766, 0.041438, 0.04038, 0.040795, 0.040456, 0.038292, 0.037835, 0.037707, 0.037526, 0.037455, 0.037309, 0.037005, 0.036911, 0.036689, 0.036713, 0.036672, 0.036701, 0.036601, 0.036215, 0.036186, 0.036204, 0.036145, 0.035993, 0.035864, 0.035584, 0.035566, 0.035502, 0.035467, 0.035332, 0.035291, 0.035151, 0.03494, 0.034929, 0.034864, 0.034701, 0.034625, 0.034525, 0.034385, 0.034426, 0.034437, 0.034408, 0.034385, 0.034355, 0.034338, 0.034145, 0.034151, 0.034168, 0.034221, 0.034198, 0.034198, 0.034075, 0.034086, 0.033987, 0.033923, 0.033975, 0.033981, 0.033934, 0.033788, 0.033776, 0.033794, 0.033747, 0.033736, 0.033817, 0.033776, 0.033841, 0.033841, 0.033759, 0.033718, 0.0337, 0.033665, 0.033677, 0.033619, 0.033659, 0.033554, 0.033543, 0.033566, 0.033595, 0.033642, 0.033589, 0.03349, 0.03356, 0.033583, 0.033513, 0.033484, 0.033484, 0.033548, 0.033531, 0.033595, 0.033595, 0.033642, 0.0337,

* Whith an error rate on 3.34% an a execution time of 21 seconds, our Gradient Boosted Trees is a better model than the Logistic Regression model that obtained an accuracy of 94.41% and a execution time of 7 minutes. 